# 自然语言处理
----------------

#### 1. 词嵌入-Word Embedding概念
1. one-hot与featurized representation  
( 1 ) 用单词在Directionary中的位置做representation的方法, 即为one-hot.   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;这种表示方法, 无法代表给出词与词之间更多的信息.例如"apple"与"oranege"同属水果.   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;而且one-hot由于用一个向量表示整个词典, 造成单词表示的向量非常大      
( 2 ) featurized representation是把单词用一系列属性表示, 用属性代替词典, 可以减小单词向量的维度. 同时可以给出更多关联信息.  

2. 何为类比(Analogy)词向量  
 使用featurized representation表示单个单词后,就可以用词向量的余弦夹角来判断词向量的相似度.   
 eg: ${ e }_{ man }-{ e }_{ women }={ e }_{ king }-{ e }_{ queen }$  
 
3. 何为embedding-matrix嵌入矩阵  
 当我们学习一个Word-Embedding时, 实际上是学习一个嵌入式矩阵.这个矩阵的每一列为一个单词的向量化形式.  
 例如一个10000个单词的词典, 每个单词用300个元素表示. 所以嵌入矩阵的维度为$R^{300*100000}$  
 
4. 学习词嵌入的过程: 有复杂算法进化到简单算法  
( 1 ) 自然语言处理就是建立一个语言模型, 来学习词嵌入.解决如何用神经网络,计算句子中下个单词是什么.  
 早期的词嵌入学习算法很复杂,而后来人们简化算法后发现效果也不错    
( 2 ) eg: I want a glass of orange  $\underline { juice }$  to go along with my cereal    
我们要学习出划线部分的单词应该为juice,此时就需要使用上下文"context".context的范围选择表现了词嵌入算法趋于简单的过程    
$context\begin{cases} 前4个单词:last\quad 4\quad words \\ 左右各四个单词:4\quad words\quad on\quad left\quad \& \quad right \\ 前一个单词:last\quad 1\quad word \\ 附近一个单词:Nearby\quad 1\quad word \end{cases}$

## 一. 词嵌入的训练方法
#### 1. word2vec
1. skip-gram模型中,我们抽取上下文context和目标词target进行配对,来构造一个监督学习问题  
( 1 ) context的抽取, 不一定要选择目标词的前4个或最近n个单词. 而是随机的选择一个词作为context. 然后在这个词的一定词距内选择另一个单词作为目标词.  
 如下图所示, 随机选择一个词作为context. target在正负10个词距内.  
 <img src="../img/skipgrams1.png" width="50%" height="50%">  
2. 现在我们假设使用一个10000词的词汇表, 我们要学习的基本监督学习问题是:   
 context c ( "orange" ) $\longrightarrow $ target t ( "juice" )  
  $\begin{cases} "orange":6257号单词 \\ "juice":4834号单词 \end{cases}$  
  即$softmax$回归学习: $\overrightarrow { { e }_{ c } } \longrightarrow softmax\longrightarrow \widehat { y } $的映射. 其中$\overrightarrow { { e }_{ c } }$表示只有6257为1,而其它位置元素为0的向量  
  
  
[注]: 由于softmax需要在10000个单词表上计算, 所以速度很慢
  
#### 2. 负采样-negative sampling
1. 负采样不再使用softmax在10000个单词上训练, 而是采用多个logistic分类器组合的形式.  
2. 步骤:   
( 1 ) 仍然采用context和target词抽取的方法: 随机的选择一个词作为context. 然后在这个词的一定词距内选择另一个单词作为目标词. 这时由于目标词在语句中出现过, 因此标记为$y=1$, 而所谓负采样, 就是在词典中, 在速记选取k个单词作为目标词, 而由于这个目标词没在语句中出现过, 因此标记为$y=0$   
( 2 ) 如下图所示: 例句" I want a glass of orange juice to go along with my cerea "   
&nbsp;&nbsp;&nbsp;&nbsp;1) 先从句子中随机选择一个context词,再在间隔内随机选择一个target词,构成正例, 对应label:1  
&nbsp;&nbsp;&nbsp;&nbsp;2) 接着进行k(k=4)个负例采样. context词不变, target词随即从字典中选择, 构成负例采样
<img src="../img/negativesample0.png" width=55% height="55%">
3. 负采样, 本质是构造一个1:k的正负例比的监督学习训练集, 其过程为  
${ \overrightarrow { e }  }_{ orange }\xrightarrow [ 下一个单词的10000个可能选择 ]{  } \begin{cases} a? \\ .. \\ juice? \\ .. \\ king? \end{cases}$  
10000个逻辑回归学习器,每个学习器给出下个单词为该单词的概率.而word2vec是要训练处10000维的巨大$softmax$回归

#### 3. GloVe
Glove不如word2vec的skipgram常用, 但是简便